In [112]:
import os, requests, time, shutil, glob, bz2, imgkit, re
from urllib.parse import urlparse, unquote
import numpy as np
from PIL import Image
from bs4 import BeautifulSoup
from multiprocessing.dummy import Pool
from src.azurekey import *

In [15]:
headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
}

def search(search_term, azureKey, offset=0):
    # print('searching using bing: "'+search_term+'"')
    search_url = "https://api.bing.microsoft.com/v7.0/search"
    subscription_key = azureKey
    assert subscription_key
    headers = {"Ocp-Apim-Subscription-Key" : subscription_key}
    params  = {"q": search_term, "responseFilter": "Webpages","count":50, "offset":offset, "safeSearch":"Strict"}
    response = requests.get(search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()
    return [v["url"] for v in search_results["webPages"]["value"]]


def save_html2jpg(url, dir_name):
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)
    req = requests.get(url, headers=headers)
    imgkit.from_string(clean(req.text), os.path.join(dir_name, str(hash(url))+".jpg"))
    
def repair_link(html, url):
    html_temp = re.sub('"/([^/"][^<"]*?)"', r'"{}\1"'.format(url), html)
    html_temp = re.sub("'/([^/'][^<']*?)'", r'"{}\1"'.format(url), html_temp)
    html_temp = re.sub("url\(/", "url({}".format(url), html_temp)
    return html_temp

def download(url, temp_dir, webpage_dir):
    save_path = webpage_dir + "/" + temp_dir
    r = requests.get(url, allow_redirects=True, headers=headers)
    file_name = os.path.split(url)[-1]
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    save_file_path = save_path + "/" + file_name
    open(save_file_path, 'wb').write(r.content)
    return temp_dir + "/" + file_name

def download_all(link_list, url, log, temp_dir="temp", webpage_dir="webpages"):
    def fetch(link):
        if link in log:
            return
        if link.startswith("http"):
            log[link] = download(link, temp_dir, webpage_dir)
        elif link.startswith("//"):
            try:
                log[link] = download("https:" + link, temp_dir, webpage_dir)
            except:
                pass
        elif link.startswith("/"):
            link_temp = os.path.split(url)[0]+link
            log[link] = download(link_temp, temp_dir, webpage_dir)
            
    modLinks=[[link_list[i],i+1] for i in range(len(link_list))]
    Pool(10).map(fetch, link_list)
    return log

def replace_all(html, log):
    html_temp = html
    for e, v in log.items():
        html_temp = re.sub(e, v, html_temp)
    return html_temp

def repair_tag(html, url, tag):
    unquoteURL = unquote(url)
    path = urlparse(unquoteURL)
    lpath = os.path.dirname(os.path.abspath(path.path))
    html_temp = re.sub('{}="(//[^h][^t][^<"]*?)"'.format(tag), r'{}="{}\1"'.format(tag, "{}:".format(path.scheme)), html)
    html_temp = re.sub('{}="(/[^h][^t][^<"]*?)"'.format(tag), r'{}="{}\1"'.format(tag, os.path.split(url)[0]), html_temp)
    html_temp = re.sub('{}="([^h][^t][^<"]*?)"'.format(tag), r'{}="{}/\1"'.format(tag, os.path.split(url)[0]), html_temp)
    html_temp = re.sub("{}='(//[^h][^t][^<']*?)'".format(tag), r"{}='{}\1'".format(tag, "{}:".format(path.scheme)), html_temp)
    html_temp = re.sub("{}='(/[^h][^t][^<']*?)'".format(tag), r"{}='{}\1'".format(tag, os.path.split(url)[0]), html_temp)
    html_temp = re.sub("{}='([^h][^t][^<']*?)'".format(tag), r"{}='{}\1'".format(tag, os.path.split(url)[0]), html_temp)
    return html_temp
                       
def repair_link(html, url):
    html_temp = repair_tag(html, url, "src")
    html_temp = repair_tag(html_temp, url, "href")
    html_temp = repair_tag(html_temp, url, "poster")
    html_temp = repair_tag(html_temp, url, "action")
    html_temp = re.sub("url\(/", "url({}/".format(os.path.split(url)[0]), html_temp)
    html_temp = re.sub("url\('/", "url('{}/".format(os.path.split(url)[0]), html_temp)
    return html_temp

In [2]:
search_results = search("electronics", azureKey, offset=0)

In [4]:
# url = "https://www.google.com/"
url = search_results[2]
req = requests.get(url, headers=headers)
print(url)

https://www.lg.com/fr


In [58]:
with open("webpages/test.html", "w") as f:
    f.write(req.text)
#     f.write(repair_link(req.text, url))

In [126]:
url = "https://www.wired.com/"
req = requests.get(url, headers=headers)
html = req.text

In [127]:
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
log = {}
log = download_all(re.findall('[^ \'()"]*?\.png', req.text), url, log, temp_dir="temp")
log = download_all(re.findall('[^ \'()"]*?\.jpg', req.text), url, log, temp_dir="temp")
log = download_all(re.findall('[^ \'()"]*?\.js', req.text), url, log, temp_dir="temp")
log = download_all(re.findall('[^ \'()"]*?\.css', req.text), url, log, temp_dir="temp")
log = download_all(re.findall('[^ \'()"]*?\.json', req.text), url, log, temp_dir="temp")

# Set Baseurl

In [155]:
url = "https://www.accenture.com/"
req = requests.get(url, headers=headers)
html = req.text
soup = BeautifulSoup(html, 'html.parser')

In [156]:
with open("webpages/test.html", "w") as f:
    f.write('<base href="{}">'.format(url))
    f.write(soup.prettify())
    with open("src/blurify.min.js", "r") as f_js:
        f.write("<script>{}</script>".format(f_js.readlines()[0]))

In [151]:
imgkit.from_file("webpages/test.html", "webpages/test.jpg", {"javascript-delay":"5000"})

Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


True

# APIFlash

In [ ]:
search_url = ""

In [ ]:
dir_name = "webpages"
url = "https://www.google.com/"
# search_url = "https://api.apiflash.com/v1/urltoimage?access_key=37bc0ab0376f4a33b4bac2e7a47776c4&height=1920&js=http%3A%2F%2F35.246.157.75%3A5000%2Fstatic%2Fjs%2Fblurify.js&url={}".format(url)
r=requests.get(search_url, stream=True)
r.raw.decode_content = True # handle spurious Content-Encoding
img = Image.open(r.raw)
img_file = os.path.join(dir_name, str(hash(url))+".jpg")
img.save(img_file)

In [ ]:
http://35.246.157.75:5000%2Fstatic%2Fjs%2Fblurify.js
http%3A%2F%2F35.246.157.75%3A5000%2Fstatic%2Fjs%2Fblurify.js